In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import csv
from tqdm import tqdm

# Load data
numInputs = 512
rowSize = 100
gridSize = 10000
x = np.zeros((numInputs, gridSize), dtype=np.float32)
y = np.zeros((numInputs, gridSize), dtype=np.float32)

with open("./smt100_ds11776/ps100_0_511.csv", "r") as f:
    reader = csv.reader(f, delimiter=",")
    it = int(0)
    for line in reader:
        # Empty row means that the set of points for the current graph
        # has ended. Update the iterator and continue
        if(len(line) == 0):
            it = it + 1
            continue
        # Skip over the starting row of each graph that only has the graph
        # number and number of steiner points
        if(len(line) == 2):
            continue
        # Coordinates of the current row
        row = int(line[1])
        column = int(line[2])
        # Fill in X and Y
        if(int(line[3]) == 0):
            x[it][rowSize*row + column] = -1.0
            y[it][rowSize*row + column] = -1.0
        else:
            y[it][rowSize*row + column] = 1.0

In [43]:
hidden_neurons = 1024
inputSize = 10000
outputSize = 10000
# Create model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(inputSize, hidden_neurons),
            nn.LeakyReLU(),
            nn.Linear(hidden_neurons, outputSize),
            nn.Hardtanh()
        )

    def forward(self, x):
        return self.layers(x)

In [75]:
# 60% 20% 20%
epochs = 20
model = Net()
my_loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0.9)
losses = []
for j in tqdm(range(epochs)):
    average_loss = 0
    for i in range(numInputs):
        x_tensor = torch.from_numpy(x[i])
        y_tensor = torch.from_numpy(y[i])
        # calls forward function on x_tensor
        output = model(x_tensor)

        loss = my_loss(output, y_tensor)
        average_loss += loss
        
        if i % 50 == 0:
            print(i, loss)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    average_loss = average_loss / 512
    losses.append(average_loss)


  0%|          | 0/20 [00:00<?, ?it/s]0 tensor(0.0142, grad_fn=<MseLossBackward>)
50 tensor(0.0146, grad_fn=<MseLossBackward>)
100 tensor(0.0143, grad_fn=<MseLossBackward>)
150 tensor(0.0145, grad_fn=<MseLossBackward>)
200 tensor(0.0142, grad_fn=<MseLossBackward>)
250 tensor(0.0137, grad_fn=<MseLossBackward>)
300 tensor(0.0143, grad_fn=<MseLossBackward>)
350 tensor(0.0138, grad_fn=<MseLossBackward>)
400 tensor(0.0143, grad_fn=<MseLossBackward>)
450 tensor(0.0148, grad_fn=<MseLossBackward>)
500 tensor(0.0144, grad_fn=<MseLossBackward>)
  5%|▌         | 1/20 [00:47<15:04, 47.60s/it]0 tensor(0.0138, grad_fn=<MseLossBackward>)
50 tensor(0.0142, grad_fn=<MseLossBackward>)
100 tensor(0.0138, grad_fn=<MseLossBackward>)
150 tensor(0.0141, grad_fn=<MseLossBackward>)
200 tensor(0.0138, grad_fn=<MseLossBackward>)
250 tensor(0.0134, grad_fn=<MseLossBackward>)
300 tensor(0.0140, grad_fn=<MseLossBackward>)
350 tensor(0.0135, grad_fn=<MseLossBackward>)
400 tensor(0.0140, grad_fn=<MseLossBackward>)
45

In [45]:
# Load in new data set
testInputs = 512
originalPoints = np.zeros(512)
x_test = np.zeros((testInputs, gridSize), dtype=np.float32)
y_test = np.zeros((testInputs, gridSize), dtype=np.float32)

with open("./smt100_ds11776/ps100_512_1023.csv", "r") as f:
    reader = csv.reader(f, delimiter=",")
    it = int(0)
    for line in reader:
        # Empty row means that the set of points for the current graph
        # has ended. Update the iterator and continue
        if(len(line) == 0):
            it = it + 1
            continue
        # Skip over the starting row of each graph that only has the graph
        # number and number of steiner points
        if(len(line) == 2):
            originalPoints[it] = line[1]
            continue
        # Coordinates of the current row
        row = int(line[1])
        column = int(line[2])
        # Fill in X and Y
        if(int(line[3]) == 0):
            x_test[it][rowSize*row + column] = -1.0
            y_test[it][rowSize*row + column] = -1.0
        else:
            y_test[it][rowSize*row + column] = 1.0



In [73]:
# Evaluate model
missedPoints = 0
numSteinerPoints = 0
accuracy = 0.0
for i in range(numInputs):
    x_tensor = torch.from_numpy(x_test[i])
    y_tensor = torch.from_numpy(y_test[i])
    # calls forward function on x_tensor
    output = model(x_tensor)
    output = output * 10
    y_pred = torch.round(output)
    # print("should be close to -1", output[3*100+17])
    # if i == 0:
    #     for num in output:
    #         print(num)

    y_pred[y_pred<0.001] = 0
    y_tensor[y_tensor<0.001] = 0
    y_pred = y_pred.type(torch.int)
    y_tensor = y_tensor.type(torch.int)
    # print(torch.bincount(y_pred))
    # print(torch.bincount(y_tensor))

    # 100 * number of missed original steiner points / number of original steiner points
    diff = y_tensor - y_pred
    # y_pred = y_pred.type(torch.int)
    # y_tensor = y_tensor.type(torch.int)
    # Only want 1s in diff
    diff[diff < 0.001] = 0
    diff = diff.type(torch.int)
    missedPoints = torch.bincount(diff)[1]
    numSteinerPoints = originalPoints[i]
    accuracy += (numSteinerPoints - missedPoints) / numSteinerPoints
accuracy = 100 * accuracy / numInputs

In [74]:
print(accuracy)

tensor(0.0602)
